In [1]:
from federation.generate_scenarios import query_time_series_data, fill_null_remove_outliers, generate_scenarios
from schema.schema import DischargeFlow
DB_CACHE_FILE = r"../.cache/interim/time_series_schema.db"
data = query_time_series_data(DB_CACHE_FILE, river="Gletsch", tables=[DischargeFlow])["DischargeFlow"]
data = fill_null_remove_outliers(data, d_time="1h", z_score=10)
data_scenarios1 = generate_scenarios(data)

In [2]:
import polars as pl
arbitrary_year = pl.col("timestamp").dt.strftime("2030-%m-%d %H:%M:%S").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False)
df = data_scenarios1.filter(pl.col("scenario")=="median").with_columns(arbitrary_year).sort("timestamp").select(["timestamp", "value"]).to_pandas().set_index("timestamp", drop=True)# .asfreq('30Min', method = 'ffill')
data_df1 = df[~df.index.duplicated()].asfreq('1H', method = 'ffill')

In [3]:
import warnings
import numpy as np
from sktime.forecasting.base import ForecastingHorizon
from pycaret.datasets import get_data
from pycaret.time_series import *
exp1 = TSForecastingExperiment()
warnings.filterwarnings("ignore")
fh = ForecastingHorizon(values=np.arange(24,48), is_relative=True)
s1 = exp1.setup(
    data=data_df1, fh=fh, fold=12, session_id=41
)

,Description,Value
0,session_id,41
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(8712, 1)"
5,Transformed data shape,"(8712, 1)"
6,Transformed train set shape,"(8665, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [4]:
from sktime.forecasting.auto_reg import AutoREG  
autoreg_sktime = AutoREG(lags=[i for i in range(24, 72)], trend="n") 
arima1 = exp1.create_model(autoreg_sktime, fit_kwargs={"fh": "{fh}"})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2030-12-17 01:00,0.0098,0.0045,0.0051,0.0058,0.0136,0.0138,-1.5311
1,2030-12-18 01:00,0.0064,0.0032,0.0033,0.0041,0.0091,0.0091,-0.4007
2,2030-12-19 01:00,0.0083,0.0042,0.0043,0.0054,0.0120,0.0118,-3.2016
3,2030-12-20 01:00,0.0083,0.0043,0.0043,0.0055,0.0119,0.0120,-1.1776
4,2030-12-21 01:00,0.0171,0.0072,0.0088,0.0092,0.0242,0.0245,-5.7597
5,2030-12-22 01:00,0.0080,0.0036,0.0041,0.0047,0.0116,0.0117,-3.4824
6,2030-12-23 01:00,0.0102,0.0046,0.0052,0.0059,0.0146,0.0147,-2.6782
7,2030-12-24 01:00,0.0508,0.0365,0.0260,0.0468,0.0609,0.0667,-0.3977
8,2030-12-25 01:00,0.0291,0.0137,0.0148,0.0175,0.0453,0.0439,-1.7738
9,2030-12-26 01:00,0.0413,0.0215,0.0210,0.0275,0.0626,0.0596,-2.4556


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
data1 = exp1.plot_model(arima1, plot="insample", return_data=True)